# Test of Curie-Meter-Rem Rule


In [1]:
import openmc
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
air = openmc.Material(name="air");
air.add_element('O',0.2);
air.add_element('N',0.8);
air.set_density('g/cm3',1.3e-3);

materials = openmc.Materials([air]);

In [3]:
s1 = openmc.Sphere(r=10.0);
s2 = openmc.Sphere(r=20.0);
s3 = openmc.Sphere(r=50.0);
s4 = openmc.Sphere(r=80.0);
s5 = openmc.Sphere(r=100.0);
s6 = openmc.Sphere(r=200.0, boundary_type='vacuum');

c1 = openmc.Cell();
c1.region = -s1;
c1.fill = air;

c2 = openmc.Cell();
c2.region = +s1 & -s2;
c2.fill = air;

c3 = openmc.Cell();
c3.region = +s2 & -s3;
c3.fill = air;

c4 = openmc.Cell();
c4.region = +s3 & -s4;
c4.fill = air;

c5 = openmc.Cell();
c5.region = +s4 & -s5;
c5.fill = air;

c6 = openmc.Cell();
c6.region = +s5 & -s6;
c6.fill = air;

root_univ = openmc.Universe();
root_univ.add_cells([c1,c2,c3,c4,c5,c6]);

geometry = openmc.Geometry();
geometry.root_universe = root_univ;

materials.export_to_xml();
geometry.export_to_xml();

In [4]:
flux = openmc.Tally(name = 'flux');
energy,dose = openmc.data.dose_coefficients('photon','AP');
dose_filter = openmc.EnergyFunctionFilter(energy,dose);
surface_filter = openmc.SurfaceFilter(s3);

flux.filters = [dose_filter,surface_filter];
flux.scores = ['current'];
tallies = openmc.Tallies([flux]);
tallies.export_to_xml();

In [5]:
point = openmc.stats.Point();

# create a discrete PMF - 50% chance each for 1.17 Mev and 1.33 MeV photons
energy_probs = openmc.stats.Discrete([1.17e6, 1.33e6],[0.5,0.5]);

source = openmc.Source(space=point,energy=energy_probs,particle='photon');

settings = openmc.Settings();
settings.run_mode = 'fixed source';
settings.batches = 100;
settings.particles = 30000;
settings.generations_per_batch = 100;
settings.source = source;

settings.export_to_xml();

In [6]:
openmc.run();

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [7]:
sp = openmc.StatePoint('statepoint.100.h5');

dose_t = sp.get_tally(name='flux');
dose_df = dose_t.get_pandas_dataframe();
dose_df.head()

,energyfunction,surface,nuclide,score,mean,std. dev.
0,103b5d3925e943,3,total,current,5.328882,0.000176


In [8]:
current = dose_df['mean'];

source_strength = 3.7e10 * 2.0; # photons per second
conv_pSv_to_rem = 1e-10;
surf_area = 4*np.pi*(50**2); # cm^2 for the given surface
time = 3600; # seconds in an hour

current = current*conv_pSv_to_rem*source_strength*time/surf_area

In [9]:
print('the dose is approximately %6.4g rem/hr' % current)

the dose is approximately  4.519 rem/hr
